<a href="https://colab.research.google.com/github/HedersonSantos/beam/blob/main/cap1/Arq1_ReadTransform_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install apache-beam[interactive]

     |████████████████████████████████| 10.1 MB 5.1 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 2.3 MB 12.4 MB/s 
     |████████████████████████████████| 508 kB 61.9 MB/s 
     |████████████████████████████████| 46 kB 3.0 MB/s 
     |████████████████████████████████| 151 kB 51.7 MB/s 
     |████████████████████████████████| 255 kB 61.4 MB/s 
     |████████████████████████████████| 793 kB 49.6 MB/s 
     |████████████████████████████████| 112 kB 45.4 MB/s 
     |████████████████████████████████| 121 kB 50.8 MB/s 
     |████████████████████████████████| 380 kB 56.7 MB/s 
     |████████████████████████████████| 1.1 MB 40.2 MB/s 
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78544 sha256=7b70bf707ee3663edb588825886da10438bfde8d648699f410a372c4399a84b7
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
  Created wheel for timeloop: filename=timeloop-1.0.2

In [2]:
!wget https://raw.githubusercontent.com/HedersonSantos/RegDatChallg/main/data/train.csv

--2022-03-28 15:47:17--  https://raw.githubusercontent.com/HedersonSantos/RegDatChallg/main/data/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1397869 (1.3M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]   1.33M  --.-KB/s    in 0.05s   

2022-03-28 15:47:18 (24.3 MB/s) - ‘train.csv’ saved [1397869/1397869]



In [19]:
#criando um registro

p1 = beam.Pipeline()
p1 | "Tupla" >> beam.Create([("Cassio",32), ("Vick",21)])| "imprime Tupla" >> beam.Map(print)
p1 | "Lista" >> beam.Create([1,2,3]) | "imprime Lista" >> beam.Map(print, end=' ')
p1.run()


1 2 3 ('Cassio', 32)
('Vick', 21)


In [31]:
import apache_beam as beam
p1 = beam.Pipeline()
#cria o objeto Pcollection para reutiliza-lo
#registros é uma PCollection
registros = (
    # as strings antes dos comandos (antes de >>) são tags. No DataFlow estas tags são utilizadas para identificar cada passo. É uma boa prática utilizá-las.
    p1
    |"Importar Dados" >> beam.io.ReadFromText("train.csv", validate=True, skip_header_lines=1)
    |"Separar por Vírgulas" >> beam.Map(lambda record: record.split(','))
    #|"Mostrar Resultados" >> beam.Map(lambda x: print(x)) 
    |"Filtrar score" >> beam.Filter(lambda record: (record[1]>'10') & (record[1]<'20'))
    |"Gravar Resultados" >> beam.io.WriteToText("teste.csv")
)
#roda a pipeline p1 (de forma assincrona)
p1.run()

In [26]:
!wget https://raw.githubusercontent.com/HedersonSantos/Noticias/main/dataset/valid.csv

--2022-03-28 16:22:44--  https://raw.githubusercontent.com/HedersonSantos/Noticias/main/dataset/valid.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 766613 (749K) [text/plain]
Saving to: ‘valid.csv’

valid.csv           100%[===================>] 748.65K  --.-KB/s    in 0.04s   

2022-03-28 16:22:45 (16.3 MB/s) - ‘valid.csv’ saved [766613/766613]



In [47]:
#FlatMap
import apache_beam as beam

palavras = ['cachorro', 'animal','Europa']

def encontrarPalavras(i):
  if i in palavras:
    return True

p1 = beam.Pipeline()
#cria o objeto Pcollection para reutiliza-lo
#registros é uma PCollection
registros = (
    # as strings antes dos comandos (antes de >>) são tags. No DataFlow estas tags são utilizadas para identificar cada passo. É uma boa prática utilizá-las.
    p1
    |"Importar Dados" >> beam.io.ReadFromText("valid.csv", validate=True, skip_header_lines=1)
    |"Quebrar por palavras" >> beam.FlatMap(lambda record: record.split(' ')) #FlatMap irá gerar uma linha a cada espaço, i.e, uma palavra em cada linha.
    |"Encontrar palavras" >> beam.Filter(encontrarPalavras)
    |"Mostrar Resultados" >> beam.Map(lambda x: print(x)) 
    |"Gravar Resultados" >> beam.io.WriteToText("teste_palavras.csv")
    
)
#roda a pipeline p1 (de forma assincrona)
p1.run()

Europa
Europa
Europa
Europa
cachorro
Europa
animal


In [48]:
p = beam.Pipeline()

#tuplas
negros = ('Machado Assis', 'Edson Arantes', 'Barack Obama')
brancos = ('FHC', 'Carlos Drummond', 'Autelina')
indios = ('Juruna', 'Raoni', 'Galdino')
#p_collections
negros_pc = p | "Criando Pcollection negros" >> beam.Create(negros)
brancos_pc = p | "Criando Pcollection brancos" >> beam.Create(brancos)
indios_pc = p | "Criando Pcollections indios" >> beam.Create(indios)

pessoas = ((negros_pc, brancos_pc, indios_pc) | beam.Flatten()) | beam.Map(print)
p.run()



Juruna
Raoni
Galdino
Machado Assis
Edson Arantes
Barack Obama
FHC
Carlos Drummond
Autelina


In [49]:
#agregação
p1 = beam.Pipeline()
#cria o objeto Pcollection para reutiliza-lo
#registros é uma PCollection
registros = (
    # as strings antes dos comandos (antes de >>) são tags. No DataFlow estas tags são utilizadas para identificar cada passo. É uma boa prática utilizá-las.
    p1
    |"Importar Dados" >> beam.io.ReadFromText("train.csv", validate=True, skip_header_lines=1)
    |"Separar por Vírgulas" >> beam.Map(lambda record: record.split(','))
    |"Filtrar score" >> beam.Filter(lambda record: (record[1]>'10') & (record[1]<'20'))
    |"Mostrar Resultados" >> beam.Map(lambda x: print(x)) 
    
    
)
#roda a pipeline p1 (de forma assincrona)
p1.run()

['403', '34.13', '0.00243', '1.757e-05', '0.00108', '0.00124', '0.00323', '0.01147', '0.105', '0.00625', '0.00658', '0.00834', '0.01874', '0.00575', '28.897', '0.46124', '0.58305', '0.12722', '0.466721605152117', '-0.824511225118934', '-1.21787456728801', '-0.432388821503959', '-1.45531978890704', '-0.280674551376605', '0.681112849927389', '0.375320150740958', '0.764563076020874', '-0.23785913776027', '-0.28219840759496']


TypeError: ignored

In [50]:
!head train.csv

"Id","score","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","V29"
403,34.13,0.00243,1.757e-05,0.00108,0.00124,0.00323,0.01147,0.105,0.00625,0.00658,0.00834,0.01874,0.00575,28.897,0.46124,0.58305,0.12722,0.466721605152117,-0.824511225118934,-1.21787456728801,-0.432388821503959,-1.45531978890704,-0.280674551376605,0.681112849927389,0.375320150740958,0.764563076020874,-0.23785913776027,-0.28219840759496
1477,19,0.0028,2.118e-05,0.00087,0.00128,0.0026,0.01575,0.137,0.00812,0.00868,0.01072,0.02435,0.009616,26.027,0.55084,0.57033,0.12869,0.805984693271499,0.0804664524854719,-0.812559263060398,1.51273059800125,0.0703305640837706,2.36875385222961,-0.256309116835819,1.31599675980002,0.0936305859555036,-0.389538267384845,1.39609478369198
5342,43.104,0.00412,2.751e-05,0.00162,0.00213,0.00487,0.02644,0.242,0.0115,0.01423,0.02545,0.0345,0.019516,23.01,0.59287,0.60478,0.26151,2.03490210521593,-0.060